In [1]:
import json
import pandas as pd
from nltk.corpus import stopwords
from unidecode import unidecode
import spacy
import plotly.express as px
import ast
from datetime import datetime
import matplotlib.pyplot as plt

nlp = spacy.load("fr_core_news_sm")

# data import

In [2]:
brands_mapping_filepath = 'data/cross_arcom_off/brands_mapping.json'
index_per_brand_filepath = 'data/cross_arcom_off/index_per_brand.json'

with open(brands_mapping_filepath, 'r') as file:
    brands_mapping = json.load(file)

with open(index_per_brand_filepath, 'r') as file:
    index_per_brand = json.load(file)

In [3]:
raw_off_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/en.openfoodfacts.org.products.csv", sep='\t')

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_55738/3360868185.py:1: DtypeWarning: Columns (0,9,15,21,22,23,29,30,31,32,33,42,44,50,54,65,70,148) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_off_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/en.openfoodfacts.org.products.csv", sep='\t')


In [4]:
raw_off_df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,abbreviated_product_name,...,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1692101569,2023-08-15T12:12:49Z,digg,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1682646029,2023-04-28T01:40:29Z,isabel626,L.casei,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050222,9999999990397,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1581171613,2020-02-08T14:20:13Z,1582644247,2020-02-25T15:24:07Z,fix-salt-bot,Fati,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3050223,9999999999,http://world-en.openfoodfacts.org/product/9999...,prepperapp,1666599417,2022-10-24T08:16:57Z,1666599418,2022-10-24T08:16:58Z,prepperapp,Duschgel - Iris,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3050224,99999999999,http://world-en.openfoodfacts.org/product/9999...,prepperapp,1683901441,2023-05-12T14:24:01Z,1683901445,2023-05-12T14:24:05Z,prepperapp,Sugo alle Melanzone,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3050225,999999999999,http://world-en.openfoodfacts.org/product/9999...,openfoodfacts-contributors,1674510561,2023-01-23T21:49:21Z,1674510561,2023-01-23T21:49:21Z,NaN,Test Product Product,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
raw_arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_55738/1379898180.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


## dataset cleaning

In [6]:
nutriscore_grade_list = ['a', 'b', 'c', 'd', 'e']

In [7]:


arcom_ids_to_keep = set()
off_ids_to_keep = set()
for brand_val in index_per_brand.values():
    arcom_ids_to_keep.update(brand_val['arcom_id'])
    off_ids_to_keep.update(brand_val['off_id'])

arcom_ids_to_keep = list(arcom_ids_to_keep)
off_ids_to_keep = list(off_ids_to_keep)

off_df = raw_off_df.loc[off_ids_to_keep]
arcom_df = raw_arcom_df.loc[arcom_ids_to_keep]

# we remove all the data from off that don't have a nutriscore grade
off_df = off_df[[grade in nutriscore_grade_list for grade in off_df['nutriscore_grade']]]

# we filter only the index in off which have a nutriscore value
for k,v in index_per_brand.items():
    index_per_brand[k]['off_id'] = [id for id in index_per_brand[k]['off_id'] if id in off_df.index]


In [8]:
mots_vides = set(stopwords.words('french'))
articles = ['le', 'la', 'les', 'un', 'une', 'de', 'du', 'des']

def remove_articles_and_stopwords(text):
    # word_tokenize raises an issue with words like M&M, where the tokens would become : ['M', '&', 'M'] and be all removed
    #tokens = word_tokenize(text, language='french')
    tokens = text.split(' ')
    mots_filtres = [mot.lower() for mot in tokens if mot.lower() not in mots_vides and mot.lower() not in articles]   
    return ' '.join(mots_filtres)

def lemmatize_french(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_punct]

def clean_off_products(row):
    product_cleaned = str(row['product_name'])
    product_cleaned = unidecode(product_cleaned)
    product_cleaned = product_cleaned.replace('%', ' percent')
    product_cleaned = product_cleaned.replace('\'', ' ')
    product_cleaned = remove_articles_and_stopwords(product_cleaned)
    row['product_name_cleaned'] = [s for s in lemmatize_french(product_cleaned) if  s.strip() != '']
    return row

def remove_brand_from_product(product, brand, caster):
    if(product.startswith(brand)):
        product = product[len(brand):]
        return product
    if(product.startswith(caster)):
        product = product[len(caster):]
    return(product)
    


def clean_arcom_products(row):
    brand = row['Marque'].lower()
    caster = row['Annonceur'].lower()
    brand = str(brand).replace('\'', ' ')
    brand = unidecode(brand)
    brand = remove_articles_and_stopwords(brand)
    product_cleaned = str(row['Produit']).lower()
    product_cleaned = product_cleaned.replace('\'', ' ')
    product_cleaned = unidecode(product_cleaned)
    product_cleaned = remove_articles_and_stopwords(product_cleaned)
    product_cleaned = remove_brand_from_product(product_cleaned, brand, caster)
    product_cleaned = product_cleaned.strip()
    row['Produit_cleaned'] = [s for s in lemmatize_french(product_cleaned) if  s.strip() != '']
    return row


In [9]:
# 4 minutes computing time
off_df = off_df.apply(clean_off_products, axis=1)
arcom_df = arcom_df.apply(clean_arcom_products, axis=1)

In [15]:


def jaccard_similarity(tokenized_str1, tokenized_str2):
    a = set(tokenized_str1)
    b = set(tokenized_str2)
    intersection = len(a.intersection(b))
    union = len(a) + len(b) - intersection
    return intersection / union if union > 0 else 0



## test on given brands : 

In [11]:
# brand labels are among those keys : 
index_per_brand.keys()

dict_keys(['2 marmottes', '2 vaches', 'a bicyclette', 'actimel', 'affinity', 'after eight', 'agri ethique france', 'almo nature', 'alpro', 'alsa', 'alter eco', 'alvalle', 'american pistachios', 'amie samba', 'amora', 'anaca 3', 'andros', 'aoste', 'apericube', 'aperivrais', 'atelier blini', 'ayam', 'bab in', 'babybel', 'babybio', 'bahlsen', 'baleine', 'banania', 'barilla', 'beautysane', 'beghin say', 'bel', 'belin', 'belvita', 'ben & jerrys', 'benco', 'benedicta', 'benenuts', 'bigard', 'biostime', 'biraghi', 'bjorg', 'bledina', 'bleu blanc coeur', 'blini', 'bn', 'bonduelle', 'bonne maman', 'bonneterre', 'bordeau chesnel', 'boulangere', 'bouquet', 'boursin', 'brennos superfoods', 'bresse bleu', 'bret', 'bridelice', 'brossard', 'buitoni', 'cacolac', 'cafe royal', 'canderel', 'candia', 'caprice dieux', 'carambar', 'carapelli', 'carre frais', 'carte noire', 'casa azzurra', 'cassegrain', 'catisfactions', 'cemoi', 'centre nat.interp.eco.lai', 'cereal', 'cerebos', 'cerise bessenay', 'cesar', '

In [12]:
off_ids = index_per_brand['2 marmottes']['off_id']
type(off_ids)

list

In [13]:
def print_similarity(brand_name):
    off_ids = index_per_brand[brand_name]['off_id']
    arcom_ids = index_per_brand[brand_name]['arcom_id']
    print('\n')
    print('off df : ')
    print(off_df.loc[off_ids,['product_name', 'product_name_cleaned']])
    print('\n')
    print('arcom df : ')
    print(arcom_df.loc[arcom_ids, ['Produit', 'Produit_cleaned']])
    print('\n')
    print('jaccard similarity : ')
    for i, row_arcom in arcom_df.loc[arcom_ids].iterrows():
        print('\n')
        print(f"arcom product name : {row_arcom['Produit']} ; {row_arcom['Produit_cleaned']} : ")
        print(' off product names : ')
        out_list = []
        for j, row_off in off_df.loc[off_ids].iterrows():
            jaccard_sim_value = jaccard_similarity(row_arcom['Produit_cleaned'], row_off['product_name_cleaned'])
            out_list.append([row_off['product_name'], row_off['product_name_cleaned'], row_off['nutriscore_grade'], jaccard_sim_value])
        out_list = sorted(out_list, key=lambda x: x[3], reverse=True)
        for row in out_list:
            print(f"    {row[0]} ; {row[1]} score = {row[2]} ; jacc = {row[3]}")

In [14]:
print_similarity('2 marmottes')



off df : 
                                         product_name  \
1360130                    INFUSION RETOUR A LA LIGNE   
1360152                              Ze  fresh  touch   
1360153           Ze fresh touch anis vert et fenouil   
1360154                                Ze Fresh touch   
1360155             Ze Fresh touch épices et thé noir   
1360071                                    Thé Génépi   
1360073                        Infusion des marmottes   
1360074                            Thym pour infusion   
1360076                              INFUSION ROMARIN   
1360078                                      Verveine   
1360079                               Infusion Enfant   
1360081                       Menthe Tilleul Verveine   
1360082                     Thé vert fleurs de Jasmin   
1360084                               Thé vert Yunnan   
1360086                             Thé Vert de Chine   
1360087                          Thé vert à la menthe   
1360088            

In [15]:
print_similarity('danone danette')



off df : 
                       product_name                    product_name_cleaned
1123688  Danette chocolat 125 g x 8  [danette, chocolat, 125, gramme, x, 8]
1123952                 Le liégeois                               [liegeoi]


arcom df : 
                   Produit     Produit_cleaned
2049       DANETTE DESSERT  [danette, dessert]
40841      DANETTE DESSERT  [danette, dessert]
5386       DANETTE DESSERT  [danette, dessert]
12810  DANETTE LE LIEGEOIS  [danette, liegeoi]
4498       DANETTE DESSERT  [danette, dessert]
4499       DANETTE DESSERT  [danette, dessert]
36885      DANETTE DESSERT  [danette, dessert]
2840   DANETTE LE LIEGEOIS  [danette, liegeoi]
5407       DANETTE DESSERT  [danette, dessert]
35     DANETTE LE LIEGEOIS  [danette, liegeoi]
5798           DANETTE POP      [danette, pop]
19750      DANETTE DESSERT  [danette, dessert]
8492       DANETTE DESSERT  [danette, dessert]
14897  DANETTE LE LIEGEOIS  [danette, liegeoi]
36786      DANETTE DESSERT  [danette, des

In [16]:
print_similarity('a bicyclette')



off df : 
                                              product_name  \
1408018                    A bicyclette Müesli abricot sec   
1408031                 jus de citron à l'avoine sans soja   
1408032                     À bicyclette framboise passion   
1408033                             Dessert mangue passion   
1408034            Spécialité végétale myrtille à l'avoine   
1407907  Spécialité végétale L'Original à l'avoine  4x100g   
1407908                           A bicyclette - Framboise   
1407909             100 % Végétal Avoine-Riz-Amande Cerise   
1407910                                A Bicyclette Citron   
1407911                               a BICYCLETTE Caramel   
1407912                            Exquis choco à l'avoine   
1408073  A bicyclette À l'avoine Chocolat et éclats de ...   
1408074                     Chocolat & eclats de noisettes   
1105882                 A bicyclette à l'avoine - Myrtille   
1407965                                       A bicyclette

In [17]:
print_similarity('actimel')



off df : 
                               product_name  \
2083359                           Actimel +   
1123917                             Actimel   
2083419                  Actimel Multifruit   
1685099                     Actimel Classic   
2083454                             Actimel   
...                                     ...   
1123267         Actimel + (x 10 Bouteilles)   
1124284                             actimel   
2421695                  Actimel fragola 0%   
1124308                             Actimel   
1123813  Actimel 0% de mg 100 g x 12 nature   

                                      product_name_cleaned  
2083359                                    [actimel, plus]  
1123917                                          [actimel]  
2083419                              [actimel, multifruit]  
1685099                                 [actimel, classic]  
2083454                                          [actimel]  
...                                                    ..

# assign a score to each product

In [16]:
# this methode finds the highest jaccard similarity between arcom and off products
# if jaccard similarity == 0 : then no score is assigned
# else if there is several products with a max jaccard distance ,we choose the socre of the most represented product
def assign_score_to_arcom_ad(off_id_list, arcom_id):
    if (len(off_id_list) == 0):
        return 'noscore1'
    arcom_product_name = arcom_df.loc[arcom_id, 'Produit_cleaned']
    similarity_off_df = off_df.loc[off_id_list, ['product_name_cleaned', 'product_name', 'nutriscore_grade']]
    similarity_off_df['similarity'] = similarity_off_df['product_name_cleaned'].apply(lambda x: jaccard_similarity(x, arcom_product_name))
    max_similarity = max(similarity_off_df['similarity'])
    if max_similarity == 0:
        return 'noscore2'
    similarity_off_df = similarity_off_df[similarity_off_df['similarity'] == max_similarity]
    return similarity_off_df['nutriscore_grade'].value_counts().idxmax()

In [12]:
for brand in index_per_brand.values():
    for arcom_id in brand['arcom_id']:
        off_id_list = brand['off_id']
        arcom_df.loc[arcom_id, 'nutriscore_inferred'] = assign_score_to_arcom_ad(off_id_list, arcom_id)

In [13]:
arcom_df['nutriscore_inferred'].value_counts()

d           1662
e           1435
noscore2    1312
b            998
c            898
noscore1     746
a            655
Name: nutriscore_inferred, dtype: int64

In [14]:
score_arcom_df = arcom_df[[score in nutriscore_grade_list for score in arcom_df['nutriscore_inferred']]]
score_arcom_df = score_arcom_df[[str(x) != 'nan' for x in score_arcom_df['clean_script']]]
score_arcom_df['script_decoded'] = score_arcom_df['Script'].apply(lambda x: unidecode(str(x)).lower())
score_arcom_df['date_formatted'] = score_arcom_df['Date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
score_arcom_df['6_month_interval'] = score_arcom_df['date_formatted'].dt.to_period('6M')

score_arcom_df['clean_script'] = score_arcom_df['clean_script'].apply(lambda x: ast.literal_eval((x)))

# export file

In [17]:
score_arcom_df.to_csv('data/cross_arcom_off/arcom_data_with_nutriscore.csv')

# brand analysis

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim import corpora
from gensim.models.ldamodel import LdaModel

## Classifier based on annotated keywords

In [23]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

In [51]:
themes = score_arcom_df['Thème'].apply(clean_themes)
score_arcom_df['clean_theme'] = themes
X = [' '.join(l) for l in themes]
y = list(score_arcom_df['nutriscore_inferred'])

In [39]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [41]:
# Initialize the Naive Bayes Classifier
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions
nb_predictions = nb_classifier.predict(X_test)

# Evaluate the classifier
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_classification_report = classification_report(y_test, nb_predictions)

print("Naive Bayes Classifier Accuracy:", nb_accuracy)
print("Naive Bayes Classifier Report:\n", nb_classification_report)

Naive Bayes Classifier Accuracy: 0.564240790655885
Naive Bayes Classifier Report:
               precision    recall  f1-score   support

           a       0.56      0.17      0.26       128
           b       0.60      0.51      0.55       192
           c       0.66      0.33      0.44       171
           d       0.51      0.72      0.60       348
           e       0.60      0.74      0.66       274

    accuracy                           0.56      1113
   macro avg       0.59      0.49      0.50      1113
weighted avg       0.58      0.56      0.54      1113



In [42]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_classifier.predict(X_test)

# Evaluate the classifier
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_classification_report = classification_report(y_test, rf_predictions)

print("Random Forest Classifier Accuracy:", rf_accuracy)
print("Random Forest Classifier Report:\n", rf_classification_report)

Random Forest Classifier Accuracy: 0.8499550763701708
Random Forest Classifier Report:
               precision    recall  f1-score   support

           a       0.93      0.81      0.87       128
           b       0.82      0.79      0.80       192
           c       0.92      0.77      0.84       171
           d       0.79      0.89      0.84       348
           e       0.88      0.91      0.89       274

    accuracy                           0.85      1113
   macro avg       0.87      0.83      0.85      1113
weighted avg       0.85      0.85      0.85      1113



In [43]:
# Initialize the Logistic Regression Classifier
lr_classifier = LogisticRegression(random_state=42)

# Train the classifier
lr_classifier.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_classifier.predict(X_test)

# Evaluate the classifier
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_classification_report = classification_report(y_test, lr_predictions)

print("Logistic Regression Classifier Accuracy:", lr_accuracy)
print("Logistic Regression Classifier Report:\n", lr_classification_report)

Logistic Regression Classifier Accuracy: 0.6433063791554358
Logistic Regression Classifier Report:
               precision    recall  f1-score   support

           a       0.70      0.47      0.56       128
           b       0.63      0.62      0.62       192
           c       0.62      0.50      0.55       171
           d       0.60      0.72      0.65       348
           e       0.71      0.74      0.72       274

    accuracy                           0.64      1113
   macro avg       0.65      0.61      0.62      1113
weighted avg       0.65      0.64      0.64      1113



/Users/thibaultrolland/anaconda3/envs/arcomenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### lda, wordcloud for scripts

In [53]:
score = 'a'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_theme'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[4.91339676e-02 8.41328315e-03 8.63025431e-03 ... 1.17380917e-03
  6.33836549e-04 1.16084446e-03]
 [4.21215445e-02 1.07329860e-02 5.63146360e-03 ... 9.00846353e-05
  9.00857631e-05 9.03940600e-05]
 [4.21768427e-02 1.28644184e-04 3.55543947e-04 ... 1.15312367e-04
  1.19678385e-04 1.15314106e-04]
 [2.07732804e-02 6.03424094e-04 2.71507502e-02 ... 1.46225604e-04
  1.49130530e-04 1.63425895e-04]
 [2.77023558e-02 4.75641480e-03 1.65361501e-02 ... 1.29345615e-04
  1.29347551e-04 1.29348118e-04]]
Topic 1: gourmandise, gout, signe de qualite autre, famille, enfant, musique celebre, joie, univers rural, nature, convivialite
Topic 2: gout, gourmandise, recette, interrogation, couple, signature sonore, complicite, nouveaute autre, choix, typographie
Topic 3: gourmandise, lien media, media internet, nature, mascotte, univers rural, dessin anime, nouveaute autre, enfant, animal autre
Topic 4: signe de qualite autre, univers culinaire, origine, musique autre, bonne humeur, effet visuel

In [54]:
score = 'b'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_theme'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[4.70566418e-04 6.46751327e-03 4.65269387e-03 ... 1.29390217e-04
  1.29382242e-04 1.29394553e-04]
 [9.28984303e-03 6.45517372e-04 7.00424705e-03 ... 1.00319594e-04
  1.00879653e-04 1.00109784e-04]
 [3.06202821e-03 1.31289440e-03 2.31297134e-04 ... 6.97118157e-05
  6.97069263e-05 6.97110518e-05]
 [9.01122461e-04 1.02790618e-04 5.19656297e-03 ... 1.22776069e-03
  4.58836817e-04 7.67790698e-05]
 [1.63264782e-03 2.41214279e-04 5.74499718e-04 ... 7.15793212e-05
  7.22170662e-05 1.14131381e-03]]
Topic 1: effet visuel, effet, joie, reference a, esoterisme, couleur, film animation, gourmandise, personne celebre autre, typographie
Topic 2: couple, logo personnage, cliche, homme, signature manuscrite, typographie, gourmandise, comportement autre, quotidien, simplicite
Topic 3: gout, gourmandise, musique celebre, enfant, environnement autre, objet personnifie, nouveaute autre, bonne humeur, tag, lien maternel
Topic 4: gout, etonnement, signe de qualite autre, origine, typographie, e

In [55]:
score = 'c'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_theme'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[1.96280386e-02 1.93218861e-04 7.43829757e-02 ... 1.85902824e-03
  8.91078962e-05 8.91054951e-05]
 [1.47208758e-02 2.30055228e-02 2.92902887e-02 ... 9.07219001e-05
  1.31614867e-03 9.05599387e-04]
 [3.98785435e-03 9.56641231e-03 2.07273737e-02 ... 1.13176342e-04
  1.13171176e-04 1.13167393e-04]
 [1.27754491e-02 1.32560590e-02 5.62015437e-02 ... 6.95624476e-05
  6.88722430e-05 6.77374119e-05]
 [1.52390829e-04 3.16414726e-03 9.53422114e-03 ... 1.42562552e-04
  1.42555582e-04 1.42554069e-04]]
Topic 1: gourmandise, gout, tag, composant special, recette, bonne humeur, femme, plaisir, petit-dejeuner, effet visuel
Topic 2: gout, gourmandise, savoir-faire autre, signe de qualite autre, enfant, origine, nature, experience, convivialite, tradition
Topic 3: typographie, couple, quotidien, signature sonore, homme, logo personnage, testimoniale anonyme, signature manuscrite, cliche, twitter
Topic 4: gourmandise, gout, snacking, nouveaute autre, objet personnifie, humour autre, signatu

In [56]:
score = 'd'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_theme'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[3.2804131e-03 5.0270809e-03 3.3779473e-03 ... 5.0882907e-05
  4.2832202e-05 1.5273970e-04]
 [2.3615325e-02 7.9760235e-03 1.0129284e-03 ... 3.1585473e-04
  3.2807986e-04 5.5471890e-05]
 [2.3600878e-03 8.5175605e-05 5.8975653e-05 ... 5.6329245e-05
  5.5874272e-05 5.5883185e-05]
 [1.4700500e-04 1.1943512e-02 2.1271850e-03 ... 4.7353169e-05
  4.7349011e-05 7.2428636e-05]
 [1.4954546e-02 8.1855811e-05 6.3707819e-05 ... 6.0658575e-05
  6.0652292e-05 1.5730927e-04]]
Topic 1: gourmandise, gout, couple, typographie, enfant, plaisir, convivialite, quotidien, complicite, humour autre
Topic 2: univers rural, gourmandise, nature, signe de qualite autre, tag, origine, naturel, animal autre, enfant, signature sonore
Topic 3: effet, bonne humeur, joie, couleur, humour autre, gout, gourmandise, convivialite, comportement autre, enfant
Topic 4: gourmandise, gout, enfant, bonne humeur, objet personnifie, tag, typographie, heros publicitaire, snacking, joie
Topic 5: gout, gourmandise, choix

In [57]:
score = 'e'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_theme'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[1.9057326e-02 5.2883064e-05 1.2975482e-03 ... 4.7910187e-05
  4.7909496e-05 4.8140158e-05]
 [1.1906116e-02 7.1201859e-05 5.8042612e-03 ... 6.7941008e-05
  6.7939363e-05 3.8841667e-04]
 [6.5211224e-04 9.5011368e-05 1.8808166e-02 ... 8.9710251e-05
  8.9652283e-05 8.9665344e-05]
 [2.9811896e-02 5.8657452e-03 2.2069735e-02 ... 6.8603214e-05
  4.1141585e-04 6.8610039e-05]
 [2.3730999e-02 4.5240275e-05 1.1366351e-02 ... 2.1228391e-04
  3.5415476e-05 4.4554374e-05]]
Topic 1: gout, gourmandise, tag, choix, savoir-faire autre, plaisir, experience, effet visuel, composant special, bonne humeur
Topic 2: gourmandise, gout, animal autre, interpellation, signature sonore, tag, joie, anglais, signature etrangere, musique celebre
Topic 3: objet personnifie, film animation, reference a, peur, typographie, conditionnement, bruitage animal, dessin anime, tag, animal autre
Topic 4: gourmandise, enfant, complicite, tag, humour autre, objet personnifie, bonne humeur, tendresse, festif, nature

## Classifier based on script

### classifier

In [19]:
X = [' '.join(l) for l in list(score_arcom_df['clean_script'])]
y = list(score_arcom_df['nutriscore_inferred'])

In [20]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

#### Naive Bayes

In [26]:
# Initialize the Naive Bayes Classifier
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions
nb_predictions = nb_classifier.predict(X_test)

# Evaluate the classifier
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_classification_report = classification_report(y_test, nb_predictions)

print("Naive Bayes Classifier Accuracy:", nb_accuracy)
print("Naive Bayes Classifier Report:\n", nb_classification_report)

Naive Bayes Classifier Accuracy: 0.8472596585804133
Naive Bayes Classifier Report:
               precision    recall  f1-score   support

           a       0.93      0.65      0.76       128
           b       0.82      0.84      0.83       192
           c       0.87      0.71      0.78       171
           d       0.81      0.93      0.86       348
           e       0.88      0.93      0.91       274

    accuracy                           0.85      1113
   macro avg       0.86      0.81      0.83      1113
weighted avg       0.85      0.85      0.84      1113



#### random forest

In [27]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_classifier.predict(X_test)

# Evaluate the classifier
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_classification_report = classification_report(y_test, rf_predictions)

print("Random Forest Classifier Accuracy:", rf_accuracy)
print("Random Forest Classifier Report:\n", rf_classification_report)

Random Forest Classifier Accuracy: 0.8876909254267745
Random Forest Classifier Report:
               precision    recall  f1-score   support

           a       0.93      0.88      0.90       128
           b       0.89      0.82      0.85       192
           c       0.92      0.82      0.87       171
           d       0.83      0.93      0.88       348
           e       0.93      0.93      0.93       274

    accuracy                           0.89      1113
   macro avg       0.90      0.88      0.89      1113
weighted avg       0.89      0.89      0.89      1113



#### logistic regression

In [28]:
# Initialize the Logistic Regression Classifier
lr_classifier = LogisticRegression(random_state=42)

# Train the classifier
lr_classifier.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_classifier.predict(X_test)

# Evaluate the classifier
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_classification_report = classification_report(y_test, lr_predictions)

print("Logistic Regression Classifier Accuracy:", lr_accuracy)
print("Logistic Regression Classifier Report:\n", lr_classification_report)

Logistic Regression Classifier Accuracy: 0.8858939802336029
Logistic Regression Classifier Report:
               precision    recall  f1-score   support

           a       0.92      0.78      0.84       128
           b       0.87      0.88      0.88       192
           c       0.87      0.80      0.84       171
           d       0.85      0.93      0.89       348
           e       0.93      0.93      0.93       274

    accuracy                           0.89      1113
   macro avg       0.89      0.87      0.88      1113
weighted avg       0.89      0.89      0.89      1113



### lda, wordcloud for scripts

In [32]:
score = 'a'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_script'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[1.39751909e-02 5.04508134e-05 6.78008888e-04 ... 5.02378898e-05
  5.61130910e-05 5.61208362e-05]
 [1.19888000e-02 6.36843033e-04 3.49760783e-04 ... 5.78010258e-05
  5.78026484e-05 5.78029249e-05]
 [5.61348442e-03 1.19417484e-04 7.62281765e-04 ... 8.15753956e-05
  8.16429820e-05 8.16535176e-05]
 [1.01711927e-02 4.09950968e-03 3.02253570e-03 ... 6.62259699e-04
  6.03787084e-05 6.03788649e-05]
 [2.18398347e-02 3.91306076e-03 3.40742245e-03 ... 6.86657222e-05
  3.92069138e-04 3.92049173e-04]]
Topic 1: pâtes, panzani, fois, non, blé, français, 4576, bon, vie, oui
Topic 2: goût, bio, lustucru, bon, soja, plaisir, bien, sojasun, viande, faire
Topic 3: pâtes, panzani, mousline, fraîches, 100, sûr, pâte, charal, découvrez, bravo
Topic 4: saurin, william, goût, légumes, bonduelle, rien, bon, bio, amour, découvrez
Topic 5: chocapic, chocolat, bon, fort, lait, délicieux, matines, oeufs, 5, goût


In [33]:
score = 'b'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_script'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[2.3031242e-03 1.7514410e-03 1.7519890e-03 ... 5.6588950e-05
  5.6588855e-05 5.6588884e-05]
 [5.7116238e-05 5.7115562e-05 5.7123052e-05 ... 5.7225705e-05
  5.7192286e-05 5.7168385e-05]
 [5.1430980e-05 5.1381106e-05 3.0751561e-04 ... 5.6342548e-04
  5.6345604e-04 5.6347798e-04]
 [2.3738443e-04 3.9584611e-05 3.9580136e-05 ... 3.7898637e-05
  3.7898713e-05 3.7898775e-05]
 [3.6191559e-05 3.5269961e-05 3.5264504e-05 ... 3.6009955e-05
  3.6009529e-05 3.6009420e-05]]
Topic 1: marie, faire, découvrez, bien, ricola, riz, goût, hollywood, taureau, zapetti
Topic 2: bien, activia, bio, intérieur, espresso, goût, matin, lait, commence, leger
Topic 3: bon, fondant, chocolat, tresor, liebig, kelloggs, chocovores, découvrez, prêts, vrais
Topic 4: café, senseo, espresso, activia, machine, faire, plaisir, calcium, découvrez, bon
Topic 5: boulangere, bon, monde, meilleur, bio, mie, simple, rendons, aidons, aident


In [34]:
score = 'c'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_script'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[4.3033669e-05 5.3749117e-04 7.1990639e-03 ... 4.2966676e-05
  4.2968673e-05 4.2967258e-05]
 [1.7356856e-03 3.6514944e-03 1.0064825e-03 ... 3.8018970e-05
  3.8020844e-05 3.8019120e-05]
 [5.8694350e-05 6.9586737e-03 4.0378589e-03 ... 4.5759341e-05
  4.5760877e-05 4.5759280e-05]
 [5.3824999e-05 2.4466498e-03 2.5655339e-03 ... 5.0961720e-05
  5.0963456e-05 5.0961731e-05]
 [7.1409042e-05 4.5260371e-04 9.8022157e-03 ... 4.2796068e-04
  1.1415330e-03 4.2795925e-04]]
Topic 1: boulangere, belvita, petit, meilleur, original, pain, simple, monde, libérez, goût
Topic 2: danette, bon, goût, plaisir, faire, tarien, 100, yaourt, chocolat, fruits
Topic 3: goût, liebig, soupe, gno, découvrez, gnocchi, non, connetable, lait, bon
Topic 4: viennois, 4, nestle, crème, chocolat, saumon, isio, tourbillon, omega, graines
Topic 5: bon, belvita, bien, soupes, cchi, 100, chocolat, special, k, déjeuner


In [35]:
score = 'd'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_script'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[2.6763217e-03 2.7458906e-05 2.7393178e-05 ... 8.1200742e-05
  2.7096263e-05 2.7096188e-05]
 [7.4437633e-04 4.5045881e-04 1.6973949e-04 ... 2.8385710e-05
  2.8383578e-05 2.8383520e-05]
 [8.2980981e-04 3.2241336e-05 3.1641699e-05 ... 3.1615640e-05
  3.1612963e-05 3.1771666e-05]
 [1.2312377e-03 2.7687011e-05 2.6800806e-05 ... 2.6802712e-05
  2.6800952e-05 2.8710107e-05]
 [7.2795531e-04 1.9333632e-04 3.1948177e-05 ... 1.2659212e-04
  1.9151575e-04 1.8900498e-04]]
Topic 1: bon, vie, kiri, goût, prince, vaches, découvrez, chocolat, hum, bio
Topic 2: rire, lu, amour, plaisir, president, goût, vache, faire, rit, meilleur
Topic 3: bon, vahine, bien, force, crème, goût, découvrez, vanille, hum, babybel
Topic 4: goût, tic, tac, frais, super, boursin, plaisir, petit, leerdammer, découvrez
Topic 5: boulangere, rians, goût, prince, meilleur, bon, laiterie, chèvre, monde, faire


In [36]:
score = 'e'
num_topics = 5

documents = list(score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]['clean_script'])
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

beta_matrix = lda_model.get_topics()

print("Beta Matrix:")
print(beta_matrix)

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id)
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in top_words])}")

Beta Matrix:
[[2.8603814e-05 3.0156432e-05 2.9007962e-05 ... 2.9252242e-05
  2.9253430e-05 6.0031802e-04]
 [3.0756826e-05 1.8406469e-04 1.8414539e-04 ... 3.0758281e-05
  3.0759060e-05 3.0757259e-05]
 [2.9331717e-05 2.9472472e-05 2.9333400e-05 ... 1.7499791e-04
  1.7499435e-04 2.9533088e-05]
 [1.9021508e-04 6.4666744e-04 6.6600734e-04 ... 3.1734762e-05
  3.1735392e-05 3.1733340e-05]
 [3.9181508e-05 6.1286388e-05 3.9184233e-05 ... 3.9183589e-05
  3.9184382e-05 3.9181570e-05]]
Topic 1: chocolat, milka, granola, découvrez, plaisir, alex, lait, biscuit, goût, tendre
Topic 2: nutella, bon, chocolat, beurre, coeur, poulain, lait, maxi, grand, vaches
Topic 3: bon, cote, goût, candia, lait, plaisir, chocolat, candyup, vraies, émotions
Topic 4: kinder, chocolat, noël, bon, cote, oui, découvrez, goûter, excellence, goût
Topic 5: lindt, lu, chocolat, creation, napolitain, découvrez, bien, gerble, faire, champs


In [ ]:
# Generate word clouds for each topic
for topic_id in range(num_topics):
    top_words = dict(lda_model.show_topic(topic_id, topn=10))  # Retrieve top words for the topic
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(top_words)
    
    # Plot word cloud for each topic
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Topic {topic_id + 1} Word Cloud')
    plt.show()

## ad count

In [1]:
score_arcom_df['nutriscore_inferred'].value_counts()

NameError: name 'score_arcom_df' is not defined

In [ ]:
for score in ['a','b','c','d','e']:
    sub_score_df = score_arcom_df[score_arcom_df['nutriscore_inferred'] == score]
    # fig = px.histogram(sub_score_df,
    #             x='date_formatted',
    #             marginal='rug', 
    #             color='Variété', title=f'repartition by sector of products graded {score}')
    # fig.show()
    fig = px.histogram(sub_score_df,
                x='Variété',
                marginal='rug', 
                color='Annonceur', title=f'Type of products graded {score.upper()}')
    fig.show()